In [2]:
import os
import random
from pathlib import Path

import pandas as pd
import torch

import matplotlib.pyplot as plt

## Общие утилиты

In [3]:
DATA_PATH = Path('data/finetune_gpt/')

DATA_PATH.mkdir(parents=True, exist_ok=True)

def seed_all(seed: int) -> None:
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)

SEED = 42

seed_all(SEED)

# Загрузка данных

[Huggingface](https://huggingface.co/datasets/d0rj/geo-reviews-dataset-2023?row=1)

In [4]:
!git clone https://github.com/eduardantonoff/fine_tuning.git

Cloning into 'fine_tuning'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17 (delta 6), reused 13 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 4.63 MiB | 11.87 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [5]:
# Преобразование данных в DataFrame
data_df = pd.read_csv('fine_tuning/data/balanced.csv')

print("Number of rows and columns in the data set:", data_df.shape)

Number of rows and columns in the data set: (14400, 3)


In [6]:
data_df.head(5)

,category,sentiment,clean_text
0,Общественное питание,Негативная,Никому не советую заказывать доставку из этого...
1,Общественное питание,Негативная,"Ужасно. Нормального подъезда нет, постоянно 3 ..."
2,Общественное питание,Негативная,Суп лапша и пюре картофельное кислое и испорче...
3,Общественное питание,Негативная,"Заказали через Яндекс еда. Всё не вкусно, Том ..."
4,Общественное питание,Негативная,Ужас. Очень долгое обслуживание и долгая подач...


In [7]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14400 entries, 0 to 14399
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category    14400 non-null  object
 1   sentiment   14400 non-null  object
 2   clean_text  14400 non-null  object
dtypes: object(3)
memory usage: 337.6+ KB


## Препроцессинг

In [8]:
work_data = data_df.dropna(subset=['clean_text'])
work_data = work_data.drop_duplicates(subset=['clean_text']).reset_index(drop=True)
work_data['clean_text'] = work_data['clean_text'].str.replace('\\n', ' ')
work_data['clean_text'][0]

'Никому не советую заказывать доставку из этого ресторана. Привезли сырые овощи на гриле, причём по три тоненькие дольки, зато головку лука здоровенный просто порезали пополам и типа поджарили, естественно весь вес дал этот сырой лук. Приправу вообще не положили, хотя в чек заботли внесли. Стоимость свинины 1600 рублей за килограмм. Это очень дорого. За 700 делают вкуснее, но администратор нам пояснил, что ценообразование связано с особыми условиями поставки высококачественного мяса. Вкус этого высококачественного мяса никого не поразил. Цена поразила. Испортили настроение. Никогда больше ничего там заказывать не будем. Заплатили 3 300 не понятно за что. Овощи отправили обратно дожаривать и попросили вернуть деньги за них и за приправу, которую не положили. Всё вернули, кроме испорченного настроения. Отзыв не заказной, а реальный. Сейчас большой выбор заведений и этот Мангал точно никогда мы больше рассматривать не будем. Работайте над собой.'

In [9]:
work_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14400 entries, 0 to 14399
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category    14400 non-null  object
 1   sentiment   14400 non-null  object
 2   clean_text  14400 non-null  object
dtypes: object(3)
memory usage: 337.6+ KB


In [10]:
unique_category = work_data['category'].unique().tolist()

unique_sentiment = work_data['sentiment'].unique().tolist()

# Модель

In [11]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from pathlib import Path

class FineTuner:
    def __init__(self,
                 model_name='ai-forever/rugpt3small_based_on_gpt2',
                 cache_dir='model_cache',
                 data_path=DATA_PATH):
        self.data_path = Path(data_path)

        # Инициализация токенизатора и модели
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name, cache_dir=str(self.data_path / cache_dir))
        self.model = GPT2LMHeadModel.from_pretrained(model_name, cache_dir=str(self.data_path / cache_dir))

    def prepare_data(self, df):
        """
        Подготовка данных для обучения
        """
        # Объединение типа проблемы и исходного текста в одну строку входных данных
        df['input'] = df.apply(
            lambda row: f"<category> {row['category']} <sentiment> {row['sentiment']} {self.tokenizer.eos_token}", axis=1
            )

        # Добавление к целевому тексту токена окончания строки
        df['output'] = df.apply(lambda row: f" <clean_text> {row['clean_text']} {self.tokenizer.eos_token}", axis=1)

        # Подготовка пути для сохранения данных
        dataset_path = self.data_path / 'train_dataset.txt'
        # Запись данных в файл
        with dataset_path.open('w', encoding='utf-8') as file:
            for input_text, target_text in zip(df['input'], df['output']):
                file.write(input_text + ' ' + target_text + '\n')
        return dataset_path

    def fine_tune(self,
                  dataset_path,
                  output_name='fine_tuned_model',
                  num_train_epochs=4,
                  per_device_train_batch_size=4,
                  learning_rate=5e-5,
                  save_steps=10_000):
        """
        Дообучение модели на заданном датасете.
        """
        train_dataset = TextDataset(
            tokenizer=self.tokenizer,
            file_path=str(dataset_path),
            block_size=256
        )

        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer, mlm=False
        )

        training_args = TrainingArguments(
            output_dir=str(self.data_path / output_name),
            overwrite_output_dir=True,
            num_train_epochs=num_train_epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            save_steps=save_steps,
            learning_rate=learning_rate,
            save_total_limit=2,
            logging_dir=str(self.data_path / 'logs'),
        )

        trainer = Trainer(
            model=self.model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=train_dataset,
        )

        trainer.train()
        # Сохранение обученной модели и токенизатора
        self.model.save_pretrained(str(self.data_path / output_name))
        self.tokenizer.save_pretrained(str(self.data_path / output_name))

In [12]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from pathlib import Path

class TextGenerator:
    def __init__(self, model_name='fine_tuned_model', data_path=DATA_PATH):
        """
        Инициализация модели и токенизатора.
        Загружаем модель и токенизатор из указанного пути.
        """
        model_path = Path(data_path) / model_name
        self.tokenizer = GPT2Tokenizer.from_pretrained(str(model_path))
        self.model = GPT2LMHeadModel.from_pretrained(str(model_path))
        self.model.eval()

    def generate_text(self,
                    category: str,
                    sentiment: str,
                    max_length=100,
                    num_return_sequences=1,
                    temperature=1.0,
                    top_k=0,
                    top_p=1.0,
                    do_sample=False):
        """
        Генерация текста на основе заданного начального текста (prompt) и параметров.

        Параметры:
        - category: Категория организации
        - sentiment: Положительная, нейтральная или негативная оценка
        - max_length: Максимальная длина сгенерированного текста.
        - num_return_sequences: Количество возвращаемых последовательностей.
        - temperature: Контролирует разнообразие вывода.
        - top_k: Если больше 0, ограничивает количество слов для выборки только k наиболее вероятными словами.
        - top_p: Если меньше 1.0, применяется nucleus sampling.
        - do_sample: Если True, включает случайную выборку для увеличения разнообразия.
        """
        # Формирование prompt
        prompt_text = f"<category> {category} <sentiment> {sentiment} {self.tokenizer.eos_token} <clean_text> "

        # Кодирование текста в формате, пригодном для модели
        encoded_input = self.tokenizer.encode(prompt_text, return_tensors='pt')

        # Генерация текстов
        outputs = self.model.generate(
            encoded_input,
            max_length=max_length + len(encoded_input[0]),
            num_return_sequences=num_return_sequences,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            do_sample=do_sample,
            no_repeat_ngram_size=2
        )

        # Декодирование результатов
        all_texts = [self.tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

        # Удаление входных данных из текстов
        prompt_length = len(self.tokenizer.decode(encoded_input[0], skip_special_tokens=True))
        trimmed_texts = [text[prompt_length:] for text in all_texts]

        # Возврат результатов в виде словаря
        return {
            "full_texts": all_texts,
            "generated_texts": trimmed_texts
        }

# Обучение

In [13]:
finetuner = FineTuner()
dataset_path = finetuner.prepare_data(work_data)
print(f'dataset_path {dataset_path}')
finetuner.fine_tune(dataset_path, output_name='fine_tuned_model_gpt_2', num_train_epochs=12)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,2.916500
1000,2.828100
1500,2.797100
2000,2.597700
2500,2.581600
3000,2.567000
3500,2.441300
4000,2.413100
4500,2.392800
5000,2.296300


# Предикт

In [14]:
unique_category[:10]

['Общественное питание', 'Магазины и Супермаркеты', 'Гостиницы и Размещение']

In [15]:
unique_sentiment[:10]

['Негативная', 'Нейтральная', 'Позитивная']

In [16]:
category = unique_category[0]
sentiment = unique_sentiment[2]

print(f'category: {category}')
print(f'sentiment: {sentiment}')

generator = TextGenerator(
    model_name='fine_tuned_model_gpt_2',
    data_path=DATA_PATH
)
generated_texts = generator.generate_text(
    category=category,
    sentiment=sentiment,
    max_length=100,
    # num_beams=3 # если несколько последовательностей
    num_return_sequences=3,
    do_sample=True,
    temperature=0.95,  # Слегка уменьшаем уверенность
    top_k=10,         # Уменьшаем количество рассматриваемых верхних k слов
    top_p=0.95        # Уменьшаем "ядерность" распределения
)
for i, text in enumerate(generated_texts['generated_texts']):
    print(f"Generated Text {i+1}: {text}")

category: Общественное питание
sentiment: Позитивная
Generated Text 1:  <cfc_periment_ Позы и напитки здесь замечательные! Персонал внимательный, обслуживание на высоком уровне, еда вкусная и разнообразная! 本好族が、ショ㄂㈂��㠂TOP! Всем хорошего дня! Хороших выходных! И удачных путешествий! С наилучшими пожеланиями, команда ресторана Club Inn Pribaltiyskaya 
Generated Text 2: ^cool  <catalogue> Отличная столовая, очень вкусные полуфабрикаты и полурыбаса. Есть возможность заказать через приложение нажав на корзину. Очень вежливый персонал. Рекомендую 本格族 ーシで。㗏ㄅ㫋㠂�㑪 中感楽㉀�cat_cafe
Generated Text 3: 中級感ツる。で㄂��㈂�㱪  <caffe> Приятное место. Очень вкусное. Вежливый персонал. Можно быстро и удобно поесть в этом месте. Есть доставка. Цены средние. Не понравилось, что в меню не было позиции ролл и стейков. 本树㠂 ㅃ㫃
